In [1]:
from src.dl.inference.inferer import Inferer
import src.dl.lightning as lightning
from src.config import CONFIG

In [2]:
lightning_model = lightning.SegModel.from_experiment(name="baseline_pannuke",version="hover")

inferer = Inferer(
    lightning_model,
    dataset="pannuke",
    patch_size=(256, 256),
    stride_size=80,
    fn_pattern="*",
    data_fold="test",
    model_weights="last",
    apply_weights=False,
    post_proc_method="cellpose"
)

In [3]:
inferer.run_inference()

Inference: /home/local/leos/Dippa/datasets/data/pannuke/test/images: 100%|██████████| 341/341 [00:36<00:00,  9.27batch/s, patches=2722/2722]


In [4]:
# inferer.soft_insts

In [5]:
# import matplotlib.pyplot as plt 
# plt.imshow(inferer.aux_maps['Breast_fold3_62'][..., 1])

In [6]:
inferer.post_process()

  0%|          | 0/2722 [00:00<?, ?it/s]/home/local/leos/Dippa/DippaEnv/lib/python3.6/site-packages/torch/nn/functional.py:3385: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn("Default grid_sample and affine_grid behavior has changed "
  1%|          | 15/2722 [00:01<03:36, 12.51it/s]/home/local/leos/Dippa/src/utils/img_utils.py:61: RuntimeWarning: divide by zero encountered in true_divide
  normed = np.clip((im / colmax), a_min=a_min, a_max=a_max)
/home/local/leos/Dippa/src/utils/img_utils.py:61: RuntimeWarning: invalid value encountered in true_divide
  normed = np.clip((im / colmax), a_min=a_min, a_max=a_max)
100%|██████████| 2722/2722 [04:05<00:00, 11.09it/s]


In [7]:
pattern_list = None
binary_scores = inferer.benchmark_insts(pattern_list=pattern_list, file_prefix="testi")
binary_scores

100%|██████████| 2722/2722 [01:21<00:00, 33.48it/s]


,AJI,AJI_plus,DICE2,PQ,SQ,DQ,inst_recall,inst_precision,splits,merges
name,,,,,,,,,,
Adrenal_gland_fold3_0,0.715410,0.734081,0.812332,0.706516,0.843894,0.837209,0.782609,0.900000,0.237587,0.260724
Adrenal_gland_fold3_1,0.702564,0.718936,0.840688,0.682747,0.837916,0.814815,0.758621,0.880000,0.198425,0.387187
Adrenal_gland_fold3_10,0.797127,0.797127,0.900026,0.721985,0.878939,0.821429,0.766667,0.884615,0.217017,0.366462
Adrenal_gland_fold3_100,0.822071,0.822071,0.968724,0.814185,0.930498,0.875000,0.875000,0.875000,0.118560,0.059920
Adrenal_gland_fold3_101,0.794434,0.794434,0.958183,0.808371,0.909418,0.888889,0.888889,0.888889,0.056084,0.167656
...,...,...,...,...,...,...,...,...,...,...
Uterus_fold3_96,0.637566,0.642674,0.676107,0.616205,0.824920,0.746988,0.704545,0.794872,0.445503,0.462230
Uterus_fold3_97,0.758090,0.780909,0.839855,0.741323,0.836977,0.885714,0.837838,0.939394,0.214778,0.325769
Uterus_fold3_98,0.710966,0.739831,0.693017,0.641455,0.846720,0.757576,0.735294,0.781250,0.382630,0.346693


In [9]:
pattern_list = None
type_scores = inferer.benchmark_types(pattern_list=pattern_list, file_prefix="testi")
type_scores = type_scores[type_scores.index.str.contains("avg")]
type_scores

100%|██████████| 2722/2722 [00:10<00:00, 266.96it/s]


,AJI,AJI_plus,DICE2,PQ,SQ,DQ,inst_recall,inst_precision,splits,merges
name,,,,,,,,,,
neoplastic_avg_for_the_set,0.548899,0.565488,0.715245,0.534175,0.782514,0.634776,0.636719,0.677485,3.496060e-01,0.360959
inflammatory_avg_for_the_set,0.323400,0.326871,0.562795,0.361782,0.540561,0.428023,0.418593,0.507244,7.890313e-02,0.124867
connective_avg_for_the_set,0.331523,0.333646,0.628527,0.349320,0.600296,0.435276,0.419109,0.529921,1.150831e-01,0.207514
dead_avg_for_the_set,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.097403e-17,0.219942
epithelial_avg_for_the_set,0.506937,0.517948,0.670908,0.503155,0.744730,0.617029,0.594649,0.696709,2.574938e-01,0.366365
